# Notebook to 

## create trains_beginning.ini from timetable.csv

## delete and recreate every delay_simulation_{}.csv file in data/delays

# moving the files from one repo to the other

In [ ]:
for file in filter(x -> endswith(x, ".csv"), readdir(path_ini))
    mv(path_ini*file,path_end*file,force=true)
end

In [1]:
using DataFrames, CSV
using IJulia

In [2]:
function get_train_begin(timetable_path::AbstractString,train_id::AbstractString)::Array{Any}
    df = DataFrame(CSV.File(timetable_path*"timetable.csv"))
    
    df=df[isequal.(df.trainid,train_id),:]
    
    if !issubset(["Begin"], df.kind) && !issubset(["Abfahrt"], df.kind)
        return [0,0,0,0]
    end
    
    df=df[findfirst(in(["Abfahrt","Beginn"]), df.kind), :]
    
    arr=[df.trainid,df.opid,df.kind,df.duetime]
    
    df= nothing
    return arr
end

get_train_begin (generic function with 1 method)

In [5]:
#getting the timetable df
timetable_path="../data/simulation_data/"
df = DataFrame(CSV.File(timetable_path*"timetable.csv"))

#getting begins array of beginnings of trains
begins=[]
train_list=unique(df.trainid)
for train in train_list
    IJulia.clear_output(true)
    println(train)
    push!(begins, get_train_begin(timetable_path,train))
end

#filter begins from zeros
b=[0,0,0,0]
filter!(x->x≠b,begins)

#printing out the file
path_out="../data/simulation_data/"
outfile = "trains_beginning.ini"
f = open(path_out*outfile, "w")
println(f,"trainid,opid,kind,duetime")
for i in 1:length(begins)
        
    println(f,begins[i][1],",",begins[i][2],",",begins[i][3],",",begins[i][4])
        
end
close(f)

SB29955


# lets redo the bash script in julia

In [7]:
function read_non_hidden_files(repo)::Vector{String}
    return filter(!startswith(".") ∘ basename, readdir(repo))
end

read_non_hidden_files (generic function with 1 method)

In [9]:
#reloading the starting df for clarity
path_out="../data/simulation_data/"
outfile = "trains_beginning.ini"
starts= DataFrame(CSV.File(path_out*outfile, comment="#"))

#creating a dict from it
start_dict=Dict((starts.trainid[i] => starts[i,2:3] for i=1:nrow(starts)) )

#defining the train list and delay list
bad_trains=String["SB22674","SB24686"]#String["SB22674","SB24686","R2246","R2265","R2248","RJ750","SB21714"]
delays=0:60:3000

#removing previous defined delay files
delays_path="../data/delays/"
files=read_non_hidden_files(delays_path)
for file in files
   rm(delays_path*file) 
end

#writing the new delay files

count=1

for train in bad_trains
    for delay in delays
        outfile = delays_path*"imposed_delay_simulation_$count.csv"
        f = open(outfile, "w")
        
        println(f,"trainid,opid,kind,duetime")
        println(f,train,",",start_dict[train].opid,",",start_dict[train].kind,",",delay)
        
        close(f)
        
        count+=1
    end
end